Build an Agent

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

In [10]:
from dotenv import load_dotenv
import os
load_dotenv()
import getpass
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = os.environ['groqe_api_key']
model = ChatGroq(model="llama3-8b-8192")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ["langchain_api_key"]


End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In [12]:
os.environ["ANTHROPIC_API_KEY"] = os.environ['anthropic_api_key']

In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [19]:
os.environ["TAVILY_API_KEY"] = os.environ['tavily_api_key']

In [21]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatGroq(model_name="llama3-8b-8192")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1hct', 'function': {'arguments': '{"query":"San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 948, 'total_tokens': 1019, 'completion_time': 0.059166667, 'prompt_time': 0.112121584, 'queue_time': 0.001059705000000008, 'total_time': 0.171288251}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3ecd5641-4ccb-4f8f-b4b1-c8598a185d75-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco'}, 'id': 'call_1hct', 'type': 'tool_call'}], usage_metadata={'input_tokens': 948, 'output_tokens': 71, 'total_tokens': 1019})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://travel.usnews.com/San_Francisco_CA/Things_To_Do/", "content": "What are some things to do in San Francis

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'I\'ve called the tool to get the current weather for you!\n\n<tool-use>{"tool_calls":[{"id":"call_2hct","type":"function","function":{"name":"weather_json"},"parameters":{"location":"San Francisco"}}]}</tool-use>\n\nPlease wait for the tool to provide the result...'}}

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1733203378, 'localtime': '2024-12-02 21:22'}, 'current': {'last_updated_epoch': 1733202900, 'last_updated': '2024-12-02 21:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 15, 'wind_dir': 'NNE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 61, 'cloud': 0, 'feelslike_c': 11.1, 'feelslike_f': 52.0, 'windchill_c': 14.3, 'windchill_f': 57.7, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 8.5, 'dewpoint_f': 47.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 9.9, 'gust_kph': 15.9}}"}, {'url': 'https://forecast.weather.gov/MapClick.php?la

In [23]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [24]:
model_with_tools = model.bind_tools(tools)

In [25]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_kg1k', 'type': 'tool_call'}]


In [26]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "weather_search"\n      },\n      "parameters": {\n        "query": "San Francisco"\n      }\n    }\n  ]\n}\n</tool-use>'}}

Create the agent

In [27]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

Run the agent

In [28]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='3ef5d8cd-dfdd-4620-b2ea-e501b23feca6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3jk1', 'function': {'arguments': '{"query":"hi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 941, 'total_tokens': 1011, 'completion_time': 0.059280854, 'prompt_time': 0.075710455, 'queue_time': 0.018460143999999998, 'total_time': 0.134991309}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cd8825fe-e266-42f7-a8dc-3a508300088e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi'}, 'id': 'call_3jk1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 941, 'output_tokens': 70, 'total_tokens': 1011}),
 ToolMessage(content='[{"url": "https://dictionary.cambridge.org/dictionary/english/hi", "content": "H

In [29]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='250d17ab-c7e3-4b8b-a25d-087b0ee098ab'),
 AIMessage(content='The current weather in San Francisco is mostly cloudy with a high of 68°F (20°C) and a low of 52°F (11°C).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1913, 'total_tokens': 1945, 'completion_time': 0.026666667, 'prompt_time': 0.364048638, 'queue_time': -0.471713061, 'total_time': 0.390715305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f2444e0-2fdc-47e8-8aed-65fb8dbddf91-0', usage_metadata={'input_tokens': 1913, 'output_tokens': 32, 'total_tokens': 1945})]

Streaming Messages

In [30]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "weather_in_sf_json"\n      },\n      "parameters": {\n        "query": "San Francisco"\n      }\n    }\n  ]\n}\n</tool-use>'}}

In [31]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Tool use failed: no tool can be called with name get_weather|

In [32]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [33]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [34]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Nice to meet you, Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 1905, 'total_tokens': 1913, 'completion_time': 0.006666667, 'prompt_time': 0.08654841, 'queue_time': -0.166629961, 'total_time': 0.093215077}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b4d7b3c-70fe-42d0-a5e0-b460d0684685-0', usage_metadata={'input_tokens': 1905, 'output_tokens': 8, 'total_tokens': 1913})]}}
----


In [35]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 965, 'total_tokens': 971, 'completion_time': 0.005, 'prompt_time': 0.111053808, 'queue_time': 0.0004360019999999909, 'total_time': 0.116053808}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-6d9199ca-bf95-43b1-8732-d317335da8ba-0', usage_metadata={'input_tokens': 965, 'output_tokens': 6, 'total_tokens': 971})]}}
----


In [36]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm afraid I don't have any information about your name.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 1908, 'total_tokens': 1922, 'completion_time': 0.011666667, 'prompt_time': 0.088039662, 'queue_time': -0.193339065, 'total_time': 0.099706329}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e8b1f5b-fc90-4c2c-891f-5bfb93f80169-0', usage_metadata={'input_tokens': 1908, 'output_tokens': 14, 'total_tokens': 1922})]}}
----
